In [ ]:
!pip install google-generativeai pandas numpy
!pip install python-dotenv
!pip install -q Streamlit

In [ ]:
import streamlit as st
import google.generativeai as genai
import pandas as pd
import numpy as np
from datetime import datetime
import os
from dotenv import load_dotenv

In [ ]:
from google.colab import userdata
apikey = userdata.get('SmartGridOpt_key')
projid = userdata.get('thrivenlive_pid')

In [ ]:
class WomensHealthChatbot:
    def __init__(self):
        """Initialize the chatbot with questions and Gemini configuration"""
        # Load environment variables
        # load_dotenv()

        # Configure Google Gemini
        GOOGLE_API_KEY = apikey
        genai.configure(api_key=GOOGLE_API_KEY)
        self.model = genai.GenerativeModel('Gemini 1.5 Flash')

        # Define questions
        self.questions = {
            'physical': {
                'text': "Hi, I'm here to help you advocate for your health and wellness. First, could you tell me about any physical health concerns you've experienced recently?",
                'options': [
                    "Persistent pain that's been dismissed",
                    "Reproductive health issues",
                    "Fatigue or sleep problems",
                    "Other symptoms",
                    "No current concerns"
                ]
            },
            'mental': {
                'text': "How would you describe your current mental wellbeing?",
                'options': [
                    "Experiencing anxiety/depression",
                    "Feeling overwhelmed/stressed",
                    "Work-life balance struggles",
                    "Generally doing okay",
                    "Prefer not to say"
                ]
            },
            'work': {
                'text': "What challenges do you face in your work environment?",
                'options': [
                    "Healthcare access issues",
                    "Workplace discrimination",
                    "Limited flexibility for health needs",
                    "No significant challenges",
                    "Not currently working"
                ]
            },
            'family': {
                'text': "What kind of support system do you have?",
                'options': [
                    "Strong family/friend support",
                    "Limited support network",
                    "Primary caregiver responsibilities",
                    "Looking to build connections",
                    "Prefer independent management"
                ]
            },
            'location': {
                'text': "How would you describe healthcare access in your area?",
                'options': [
                    "Limited access to specialists",
                    "Long waiting times",
                    "Transportation barriers",
                    "Good healthcare access",
                    "Unsure about available resources"
                ]
            }
        }

         # Initialize session state
        if 'current_step' not in st.session_state:
            st.session_state.current_step = 0
        if 'answers' not in st.session_state:
            st.session_state.answers = {}
        if 'recommendations' not in st.session_state:
            st.session_state.recommendations = None


    def generate_prompt(self, answers):
        """Generate a prompt for Gemini based on user answers"""
        prompt = f"""
        As a healthcare advocate specialized in women's health, generate personalized recommendations based on the following responses:

        Physical Health: {answers.get('physical', 'Not specified')}
        Mental Health: {answers.get('mental', 'Not specified')}
        Work Environment: {answers.get('work', 'Not specified')}
        Support System: {answers.get('family', 'Not specified')}
        Healthcare Access: {answers.get('location', 'Not specified')}

        Please provide:
        1. Immediate action steps for addressing health concerns
        2. Self-advocacy strategies
        3. Resources and support options
        4. Long-term wellness recommendations

        Format the response in clear, actionable bullet points.
        Focus on practical solutions that consider potential healthcare discrimination.
        Maintain an empathetic and supportive tone.
        """
        return prompt

    def get_gemini_recommendations(self, answers):
        """Get personalized recommendations from Gemini"""
        try:
            prompt = self.generate_prompt(answers)
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            st.error(f"Error generating recommendations: {str(e)}")
            return None

    def save_data(self, answers, recommendations):
        """Save user data and recommendations to CSV"""
        try:
            df = pd.DataFrame([{
                'timestamp': datetime.now(),
                **answers,
                'recommendations': recommendations
            }])
            df.to_csv('health_data.csv', mode='a', header=not os.path.exists('health_data.csv'), index=False)
        except Exception as e:
            st.warning(f"Could not save data: {str(e)}")

    def run(self):
        """Main chatbot interface"""
        st.title("Women's Health Advocacy Chatbot")

        if st.session_state.recommendations is None:
            # Display current question
            if st.session_state.current_step < len(self.questions):
                question_key = list(self.questions.keys())[st.session_state.current_step]
                question = self.questions[question_key]

                st.write(question['text'])
                selected_option = st.radio("Choose an option:", question['options'], key=f"radio_{st.session_state.current_step}")

                if st.button("Next"):
                    st.session_state.answers[question_key] = selected_option
                    st.session_state.current_step += 1
                    st.rerun()

                # Show progress
                st.progress(st.session_state.current_step / len(self.questions))

            # Generate recommendations when all questions are answered
            elif st.session_state.current_step == len(self.questions):
                with st.spinner("Generating personalized recommendations..."):
                    recommendations = self.get_gemini_recommendations(st.session_state.answers)
                    if recommendations:
                        st.session_state.recommendations = recommendations
                        self.save_data(st.session_state.answers, recommendations)
                        st.rerun()

        else:
            # Display recommendations
            st.subheader("Your Personalized Wellness Plan")
            st.markdown(st.session_state.recommendations)

            if st.button("Start Over"):
                st.session_state.current_step = 0
                st.session_state.answers = {}
                st.session_state.recommendations = None
                st.rerun()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Streamlit app code
def main():
    st.set_page_config(page_title="Women's Health Advocate", page_icon="❤️")

    # Add custom CSS
    st.markdown("""
        <style>
        .stRadio > label {
            font-size: 16px;
            padding: 10px;
        }
        .stButton button {
            width: 100%;
            margin-top: 20px;
        }
        </style>
    """, unsafe_allow_html=True)

    # Initialize and run chatbot
    chatbot = WomensHealthChatbot()
    chatbot.run()

#Run the Streamlit app
if __name__ == "__main__":
    main()